In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager


# use GeckoDriver manager to access firefox page via gecko
executable = FirefoxService(GeckoDriverManager().install())

driver = webdriver.Firefox(service=executable)

[WDM] - Downloading: 16.9kB [00:00, 8.64MB/s]                   


In [61]:
url_base = "https://www.indeed.com/jobs?q="
pg = 0

position_type = ['software engineer', 'full time']
search = '+'.join(' '.join(position_type).split())
search

url_search = url_base + search

In [153]:
all_titles = []
all_companies = []
all_loc = []
all_salary = []
all_type = []
all_text = []

page = driver.get(url_search)   
driver.implicitly_wait(10)

# pull job cards, excluding final blank card

to_click = [1,2,3,4,5,7,8,9,10,11,13,14,15,16,17]

try:
    date_sort = driver.find_element(By.CSS_SELECTOR, ".jobsearch-DesktopSort > span:nth-child(2) > a:nth-child(2)")
    date_sort.click()
except:
    pass

for card in to_click:
    card_search = ".jobsearch-ResultsList > li:nth-child({})".format(card) 
    job_card = driver.find_element(By.CSS_SELECTOR,card_search)
    job_card.click()
    driver.implicitly_wait(5)
    
    desc = driver.find_element(By.CSS_SELECTOR, ".jobsearch-JobComponent-description")
    job_text = desc.text.split("Full Job Description\n")[1]
    all_text.append(job_text)
    
    soup = BeautifulSoup(job_card.get_attribute('innerHTML'), 'html.parser')

    # get html from root node
    soup_children = list(soup.children)
    html = soup_children[0]    

    # Get head from html 
    html_children = list(html.children)
    head = html_children[0]
    
    # get job card stats
    job_title = head.find('h2').text
    company = head.find("span", {"class":"companyName"}).text
    location = head.find("div",{"class":"companyLocation"}).text
    
    attr = head.find("div",{"class":"attribute_snippet"}).text
    if sum(map(str.isdigit, attr)) > 2:
        salary = attr
    else:
        salary = 'n/a'
    
    all_titles.append(job_title)
    all_companies.append(company)
    all_loc.append(location)
    all_salary.append(salary)
      

In [154]:
criteria = ['title','company','location','salary','description']
df = pd.DataFrame(columns=criteria)

df['title'] = all_titles
df['company'] = all_companies
df['location'] = all_loc
df['salary'] = all_salary
df['description'] = all_text

df

,title,company,location,salary,description
0,Software Engineer I,Staples,"Framingham, MA 01702+1 location",n/a,Software Engineer I\nStaples is business to bu...
1,Software Developer,Grant Street Group,Remote,n/a,Do you love working with a high caliber team t...
2,Software Engineer,Red Hat Software,"Raleigh, NC 27601 (Central area)+5 locations",n/a,About the job:\nWe are looking for a Software ...
3,Software Engineer Job Ref #: 224422,Concentrix Catalyst,"Remote in Omaha, NE 68102",n/a,Overview\nConcentrix CVG Customer Management G...
4,Software Engineer,Skyline Products,+2 locationsRemote,"$85,000 - $100,000 a year",PriceAdvantage is recruiting for a remote Soft...
5,Backend Staff Software Engineer,Next College Student Athlete,Remote,n/a,Job Description\nWho We Are:\nIMG Academy is t...
6,Software Engineer - Associate (Power Platform),UPMC,+3 locationsRemote,n/a,Description\nPurpose:\nUnder the direction of ...
7,Software Engineer,Remesh,"Remote in San Francisco, CA",n/a,"Remesh - Cleveland, OH or Remote\nFull-time\n1..."
8,Software Engineer,Regions,"Birmingham, AL 35203+7 locations",n/a,Thank you for your interest in a career at Reg...
9,Full Stack Developer (React / Node JS),American Recruiting & Consulting Group,+2 locationsRemote,n/a,REACTJS/NODEJS DEVELOPER – REMOTE\n\nARC Group...
